In [1]:
#default_exp haystack_code_search

In [24]:
#export
import pprint
import numpy as np
import pandas as pd
import requests
import torch
from sklearn import metrics
from nltk import tokenize
from operator import itemgetter

from haystack import Finder
from haystack.database.elasticsearch import ElasticsearchDocumentStore
from haystack.database.memory import InMemoryDocumentStore

from haystack.retriever.dense import EmbeddingRetriever
from haystack.utils import print_answers
from pytorch_hackathon import rss_feeds

import seaborn as sns

In [69]:
pd.set_option('max_colwidth', 100)

In [25]:
cm = sns.light_palette("green", as_cmap=True)

In [3]:
%cd ..

/home/kuba/Projects/pytorch_hackathon


In [4]:
!ls data

feeds.txt  topics.txt  zsl_feed_results.csv


In [5]:
rss_feed_urls = list(pd.read_table('data/feeds.txt', header=None).iloc[:,0].values)

In [6]:
feed_df = rss_feeds.get_feed_df(rss_feed_urls)

100%|██████████| 16/16 [00:09<00:00,  1.69it/s]
/home/kuba/Projects/pytorch_hackathon/pytorch_hackathon/rss_feeds.py:63: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 63 of the file /home/kuba/Projects/pytorch_hackathon/pytorch_hackathon/rss_feeds.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  feed_df['text'] = feed_df['summary'].apply(lambda s: bs4.BeautifulSoup(s).text)


In [7]:
#export

use_gpu = torch.cuda.is_available()

In [8]:
pretty_print = pprint.PrettyPrinter(indent=2).pprint

In [9]:
feed_df.head()

,title,title_detail,links,link,summary,summary_detail,id,guidislink,tags,text,...,published_parsed,comments,authors,author,author_detail,updated,updated_parsed,content,href,media_thumbnail
0,Norm-in-Norm Loss with Faster Convergence and Better Performance for Image Q...,"{'type': 'text/plain', 'language': None, 'base': 'https://us-east1-ml-feeds....","[{'rel': 'alternate', 'type': 'text/html', 'href': 'https://paperswithcode.c...",https://paperswithcode.com/paper/norm-in-norm-loss-with-faster-convergence-and,"Experiments on two relevant datasets (KonIQ-10k and CLIVE) show that, compar...","{'type': 'text/html', 'language': None, 'base': 'https://us-east1-ml-feeds.c...",https://paperswithcode.com/paper/norm-in-norm-loss-with-faster-convergence-and,False,"[{'term': 'Image quality assessment', 'scheme': None, 'label': None}]","Experiments on two relevant datasets (KonIQ-10k and CLIVE) show that, compar...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Prototype Mixture Models for Few-shot Semantic Segmentation,"{'type': 'text/plain', 'language': None, 'base': 'https://us-east1-ml-feeds....","[{'rel': 'alternate', 'type': 'text/html', 'href': 'https://paperswithcode.c...",https://paperswithcode.com/paper/prototype-mixture-models-for-few-shot,Few-shot segmentation is challenging because objects within the support and ...,"{'type': 'text/html', 'language': None, 'base': 'https://us-east1-ml-feeds.c...",https://paperswithcode.com/paper/prototype-mixture-models-for-few-shot,False,"[{'term': 'Few-shot semantic segmentation', 'scheme': None, 'label': None}, ...",Few-shot segmentation is challenging because objects within the support and ...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Automatic Failure Recovery and Re-Initialization for Online UAV Tracking wit...,"{'type': 'text/plain', 'language': None, 'base': 'https://us-east1-ml-feeds....","[{'rel': 'alternate', 'type': 'text/html', 'href': 'https://paperswithcode.c...",https://paperswithcode.com/paper/automatic-failure-recovery-and-re,Current unmanned aerial vehicle (UAV) visual tracking algorithms are primari...,"{'type': 'text/html', 'language': None, 'base': 'https://us-east1-ml-feeds.c...",https://paperswithcode.com/paper/automatic-failure-recovery-and-re,False,"[{'term': 'Visual tracking', 'scheme': None, 'label': None}]",Current unmanned aerial vehicle (UAV) visual tracking algorithms are primari...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Vision Meets Wireless Positioning: Effective Person Re-identification with R...,"{'type': 'text/plain', 'language': None, 'base': 'https://us-east1-ml-feeds....","[{'rel': 'alternate', 'type': 'text/html', 'href': 'https://paperswithcode.c...",https://paperswithcode.com/paper/vision-meets-wireless-positioning-effective,Existing person re-identification methods rely on the visual sensor to captu...,"{'type': 'text/html', 'language': None, 'base': 'https://us-east1-ml-feeds.c...",https://paperswithcode.com/paper/vision-meets-wireless-positioning-effective,False,"[{'term': 'Person re-identification', 'scheme': None, 'label': None}]",Existing person re-identification methods rely on the visual sensor to captu...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,T-GD: Transferable GAN-generated Images Detection Framework,"{'type': 'text/plain', 'language': None, 'base': 'https://us-east1-ml-feeds....","[{'rel': 'alternate', 'type': 'text/html', 'href': 'https://paperswithcode.c...",https://paperswithcode.com/paper/t-gd-transferable-gan-generated-images,"First, we train the teacher model on the source dataset and use it as a star...","{'type': 'text/html', 'language': None, 'base': 'https://us-east1-ml-feeds.c...",https://paperswithcode.com/paper/t-gd-transferable-gan-generated-images,False,NaN,"First, we train the teacher model on the source dataset and use it as a star...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
tokenize.wordpunct_tokenize

<bound method RegexpTokenizer.tokenize of WordPunctTokenizer(pattern='\\w+|[^\\w\\s]+', gaps=False, discard_empty=True, flags=<RegexFlag.UNICODE|DOTALL|MULTILINE: 56>)>

In [11]:
#export


def setup_document_store_with_retriever(model_name, df, text_col='text', max_document_length=256, use_gpu=use_gpu, quantize_model=True):
    embedding_col = text_col + '_emb'
    document_store = InMemoryDocumentStore(
        embedding_field=embedding_col,
    )
    retriever = EmbeddingRetriever(
        document_store=document_store,
        embedding_model=model_name,
        use_gpu=use_gpu)
    

    if not use_gpu and quantize_model:
        quantized_model = torch.quantization.quantize_dynamic(
            retriever.embedding_model.model,
            {torch.nn.Linear}, dtype=torch.qint8
        )

    article_embeddings = retriever.embed_queries(
        texts=[
            ' '.join(tokenize.wordpunct_tokenize(text)[:max_document_length])
            for text in df[text_col] 
        ]
    )

    df[embedding_col] = article_embeddings
    document_store.write_documents(df.to_dict(orient='records'))
    return document_store, retriever

In [12]:
article_texts = feed_df['text']

In [13]:
model_name = "deepset/sentence_bert"
document_store, retriever = setup_document_store_with_retriever(model_name, feed_df, 'text', use_gpu)

08/13/2020 18:29:02 - INFO - haystack.retriever.dense -   Init retriever using embeddings of model deepset/sentence_bert
08/13/2020 18:29:02 - INFO - farm.utils -   device: cuda n_gpu: 1, distributed training: False, automatic mixed precision training: None
08/13/2020 18:29:02 - INFO - farm.infer -   Could not find `deepset/sentence_bert` locally. Try to download from model hub ...
08/13/2020 18:29:04 - WARNING - farm.modeling.language_model -   Could not automatically detect from language model name what language it is. 
	 We guess it's an *ENGLISH* model ... 
	 If not: Init the language model by supplying the 'language' param.
08/13/2020 18:29:10 - INFO - farm.utils -   device: cuda n_gpu: 1, distributed training: False, automatic mixed precision training: None
Inferencing Samples: 100%|██████████| 74/74 [00:09<00:00,  7.46 Batches/s]


In [14]:
feed_df.iloc[1]['text']

'Few-shot segmentation is challenging because objects within the support and query images could significantly differ in appearance and pose. Code: https://github.com/Yang-Bob/PMMs'

In [15]:
#export


def doc_to_dict(doc):
    d = {}
    d['text'] = doc.text
    d['title'] = doc.meta['title']
    d['score'] = doc.query_score
    return d

In [61]:
topic_strings = pd.read_table('data/topics.txt', header=None).iloc[:,0].values

In [62]:
print('\n'.join(topic_strings))

deep learning
natural language processing
computer vision and image processing
statistics
implementation
visualization
industry
software engineering
reddit question
arxiv
cloud computing
deployment
competitions
business
business intelligence


In [63]:
topic_query_strings = [
    'text is about {}'.format(topic)
    for topic in topic_strings
]

In [64]:
raw_results = retriever.retrieve(
    topic_query_strings[1]
)

Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00, 15.79 Batches/s]


In [65]:
#export


def get_scored_df(raw_results, topic_strings):
    topic_query_strings = [
        'text is about {}'.format(topic)
        for topic in topic_strings
    ]
    
    results = [
        doc_to_dict(doc)
        for doc in raw_results 
    ]
    result_embeddings = np.array([
        doc.meta['text_emb']
        for doc in raw_results
    ]).astype('float32')
    topic_query_embeddings = np.array(retriever.embed_passages(
        list(topic_strings)
    )).astype('float32')
    
    scores_df = pd.DataFrame({})
    scores_df['title'] = list(map(itemgetter('title'), results))
    scores_df['text'] = list(map(itemgetter('text'), results))

    scores = pd.DataFrame(metrics.pairwise.cosine_similarity(
        result_embeddings,
        topic_query_embeddings
    ))
    scores.columns = topic_strings

    scores_df = pd.concat(
        [scores_df, scores],
        axis=1
    )
    return scores_df

In [66]:
scores_df = get_scored_df(raw_results, topic_strings)

Inferencing Samples: 100%|██████████| 4/4 [00:00<00:00,  7.12 Batches/s]


In [67]:
len(set(scores_df.columns))

17

In [70]:
scores_df.style.background_gradient(cmap=cm)

In [ ]:
[doc.text for doc in raw_results]

In [ ]:
[doc.text for doc in raw_results]

In [ ]:
scores_df

In [ ]:
import ktrain

zsl_clf = ktrain.text.ZeroShotClassifier(device='cuda')

In [ ]:
scores = [
    zsl_clf.predict(d['text'], topic_strings=topic_strings, max_length=256, include_labels=True)
    for d in results
]

In [ ]:
results[0]

In [ ]:
results_df = pd.DataFrame.from_records(
    {**result, **dict(doc_scores)}
    for (result, doc_scores) in zip(results, scores)
)

In [ ]:
results_df